# Loading Libraries

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time

import tensorflow as tf
print("Num GPUs Available TF: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import cv2
count = cv2.cuda.getCudaEnabledDeviceCount()
print("Num GPUs Available OpenCV: ", count)

Num GPUs Available TF:  1
Num GPUs Available OpenCV:  1


This cell should print: <br />

    Num GPUs Available TF:  1
    Num GPUs Available OpenCV:  1
If this is not the case then GPU acceleration is not active and Cuda cores are not configured properly.

# Loading the Models

In [8]:
# Loading trained CNN model to use it later when classifying from 4 groups into one of 43 classes
from tensorflow.keras.models import load_model
model = load_model("TSC_final_updated.h5")

# Path to Weights and Configuration File
path_to_weights = 'yolov3_training_last.weights'
path_to_cfg = 'yolov3_ts_test.cfg'

# Loading trained YOLO v3 weights and cfg configuration file by 'dnn' library from OpenCV
network = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)

# To use with GPU
network.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
network.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [9]:
# Getting names of all YOLO v3 layers
layers_all = network.getLayerNames()

# Getting only detection YOLO v3 layers that are 82, 94 and 106
layers_names_output = [layers_all[i - 1] for i in network.getUnconnectedOutLayers()]

print(layers_names_output)  # ['yolo_82', 'yolo_94', 'yolo_106']

['yolo_82', 'yolo_94', 'yolo_106']


In [10]:
labels= pd.read_pickle("updated_labels.pickle")

# Minimum probability to eliminate weak detections
probability_minimum = 0.2

# Setting threshold to filtering weak bounding boxes by non-maximum suppression
threshold = 0.2

# Generating colours for bounding boxes
colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# Check point
print(type(colours))  # <class 'numpy.ndarray'>
print(colours.shape)  # (43, 3)
print(colours[0])

<class 'numpy.ndarray'>
(43, 3)
[ 68  57 210]


# The Application Cells

## 3 Modes are available: <br />

    Mode 1: Real-time with no video saving.
    Mode 2: Real-time with video saving.
    Mode 3: Video playback (Using the Application on an already recorded video) with video saving

Each cell contains the code for 1 mode. <br />
Execute only the cell for your inended purpose.  

### Mode 1
Realtime with no video saving.

In [12]:
# MODE 1 Cell

%matplotlib inline

# Setting the video capture device
video = cv2.VideoCapture(0)

# Writer that will be used to write processed frames
writer = None

# Variables for spatial dimensions of the frames
h, w = None, None

# Setting default size of plots
plt.rcParams['figure.figsize'] = (3, 3)

# Variable for counting total amount of frames
f = 0

# Variables for counting processing time
t = 0
t2 = 0
t_ping_prev=0

# Warmup
frame = np.random.randint(255, size=(416, 416, 3), dtype=np.uint8)
blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
for i in range(3):
    network.setInput(blob)
    detections = network.forward(layers_names_output)


# Catching frames in the loop
while True:
    # Capturing frames one-by-one
    ret, frame = video.read()

    #Calculating FPS
    t_ping= time.time()
    ping= 1/(t_ping - t_ping_prev)
    t_ping_prev= t_ping

    # If the frame was not retrieved
    if not ret:
        break
       
    # Getting spatial dimensions of the frame for the first time
    if w is None or h is None:
        # Slicing two elements from tuple
        h, w = frame.shape[:2]

    # Blob from current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # Forward pass with blob through output layers
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increasing counters
    f += 1
    t += end - start

    # Lists for detected bounding boxes, confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Filtring Low probability predictions with numpy function to optimize performance
        # 1st order Optimazation
        opt= result[:,5:]
        opt= np.max(opt, axis=1)
        opt2= np.where(opt> probability_minimum )[0]
        # Going through all detections after the optimazation from current output layer 
        for obj in opt2:
            detected_objects= result[obj]
            confidence_current= opt[obj]
            class_current= np.where(detected_objects==confidence_current)[0][0]-5

            # Scaling bounding box coordinates to the initial frame size
            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            # Getting top left corner coordinates
            x_center, y_center, box_width, box_height = box_current
            # Scaler to tune box size with precision
            Box_Scaler= 1.25
            box_width= box_width * Box_Scaler 
            box_height= box_height * Box_Scaler 
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            # Adding results into prepared lists
            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)
                


    # Implementing non-maximum suppression of given bounding boxes
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    # Genarating a copy from the frame in order to print the labels on this copy
    frame_copy= frame.copy()

    # Checking if there is any detected object been left
    if len(results) > 0:
        dimentions=[]
        # Variable for counting the number of True Signs in the frame
        nbre=0
        # Array containing the signs that will be fed to the classification model
        to_classify= np.array([])
        # Going through indexes of results
        for i in results.flatten():
            # Bounding box coordinates, its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            
            # Cut fragment with Traffic Sign
            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
            # print(c_ts.shape)
            
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                nbre+=1
                # Saving the information about the signs in a list
                dimentions.append((x_min, y_min, box_width, box_height, class_numbers[i]))

                # Getting preprocessed blob with Traffic Sign of needed shape
                blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts = blob_ts.transpose(0, 2, 3, 1)

                # Appending the Signs-to-classify array 
                if nbre==1:
                    to_classify= blob_ts
                else:
                    to_classify= np.concatenate((to_classify, blob_ts), axis=0)
                

        if nbre!=0:
            # Obtaing the classification result
            # 2nd order optimazation
            scores= model(to_classify, training=False)

            # Itirating the final predictions in order to print the results 
            for i in range(nbre):

                # Scores is given for image with 43 numbers of predictions for each class
                # Getting only one class with maximum value
                prediction = np.argmax(scores[i])

                # Reobtaing the sign dimentions 
                x_min, y_min, box_width, box_height= dimentions[i][0], dimentions[i][1], dimentions[i][2], dimentions[i][3] 

                # Colour for current bounding box
                colour_box_current = colours[dimentions[i][4]].tolist()

                # Drawing bounding box on the original current frame
                Box_Rev_Scaler= 1/Box_Scaler
                x_min= round(x_min + box_width*(1-Box_Rev_Scaler)/2)
                y_min= round(y_min + box_height*(1-Box_Rev_Scaler)/2)
                box_width= round(box_width*Box_Rev_Scaler)
                box_height= round(box_height*Box_Rev_Scaler)
                cv2.rectangle(frame_copy, (x_min, y_min),
                                (x_min + box_width, y_min + box_height),
                                colour_box_current, 2)

                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: [{:.3f}]({:.3f})'.format(labels[prediction], 
                                                        scores[i][prediction],
                                                        confidences[i])

                # Putting text with label and confidence on the original image
                cv2.putText(frame_copy, text_box_current, (x_min - 50 , y_min - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.45, colour_box_current, 2)



    # Printing FPS
    cv2.putText(frame_copy, str(round(ping)), (5 , 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, [51, 204, 51], 3)

    # Display the resulting frame. The output window resolution is 1280x720
    cv2.imshow('Video', cv2.resize(frame_copy, (1280, 720)))

    end2 = time.time()
    t2 += end2 - start

    # Press Q to close the result window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Releasing video reader and open windows
video.release()
cv2.destroyAllWindows()

### Mode 2: 
Realtime with video saving.

In [ ]:
# MODE 2 Cell

# Fill Video output name
Result_Video_Name= "FILL ME"

%matplotlib inline

# Setting the video capture device
video = cv2.VideoCapture(0)

# Writer that will be used to write processed frames
writer = None

# Variables for spatial dimensions of the frames
h, w = None, None

# Setting default size of plots
plt.rcParams['figure.figsize'] = (3, 3)

# Variable for counting total amount of frames
f = 0

# Variables for counting processing time
t = 0
t2 = 0
t_ping_prev=0

# Warmup
frame = np.random.randint(255, size=(416, 416, 3), dtype=np.uint8)
blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
for i in range(3):
    network.setInput(blob)
    detections = network.forward(layers_names_output)


# Catching frames in the loop
while True:
    # Capturing frames one-by-one
    ret, frame = video.read()

    #Calculating FPS
    t_ping= time.time()
    ping= 1/(t_ping - t_ping_prev)
    t_ping_prev= t_ping

    # If the frame was not retrieved
    if not ret:
        break
       
    # Getting spatial dimensions of the frame for the first time
    if w is None or h is None:
        # Slicing two elements from tuple
        h, w = frame.shape[:2]

    # Blob from current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # Forward pass with blob through output layers
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increasing counters
    f += 1
    t += end - start

    # Lists for detected bounding boxes, confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Filtring Low probability predictions with numpy function to optimize performance
        # 1st order Optimazation
        opt= result[:,5:]
        opt= np.max(opt, axis=1)
        opt2= np.where(opt> probability_minimum )[0]
        # Going through all detections after the optimazation from current output layer 
        for obj in opt2:
            detected_objects= result[obj]
            confidence_current= opt[obj]
            class_current= np.where(detected_objects==confidence_current)[0][0]-5

            # Scaling bounding box coordinates to the initial frame size
            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            # Getting top left corner coordinates
            x_center, y_center, box_width, box_height = box_current
            # Scaler to tune box size with precision
            Box_Scaler= 1.25
            box_width= box_width * Box_Scaler 
            box_height= box_height * Box_Scaler 
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            # Adding results into prepared lists
            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)
                


    # Implementing non-maximum suppression of given bounding boxes
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    # Genarating a copy from the frame in order to print the labels on this copy
    frame_copy= frame.copy()

    # Checking if there is any detected object been left
    if len(results) > 0:
        dimentions=[]
        # Variable for counting the number of True Signs in the frame
        nbre=0
        # Array containing the signs that will be fed to the classification model
        to_classify= np.array([])
        # Going through indexes of results
        for i in results.flatten():
            # Bounding box coordinates, its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            
            # Cut fragment with Traffic Sign
            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
            # print(c_ts.shape)
            
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                nbre+=1
                # Saving the information about the signs in a list
                dimentions.append((x_min, y_min, box_width, box_height, class_numbers[i]))

                # Getting preprocessed blob with Traffic Sign of needed shape
                blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts = blob_ts.transpose(0, 2, 3, 1)

                # Appending the Signs-to-classify array 
                if nbre==1:
                    to_classify= blob_ts
                else:
                    to_classify= np.concatenate((to_classify, blob_ts), axis=0)
                

        if nbre!=0:
            # Obtaing the classification result
            # 2nd order optimazation
            scores= model(to_classify, training=False)

            # Itirating the final predictions in order to print the results 
            for i in range(nbre):

                # Scores is given for image with 43 numbers of predictions for each class
                # Getting only one class with maximum value
                prediction = np.argmax(scores[i])

                # Reobtaing the sign dimentions 
                x_min, y_min, box_width, box_height= dimentions[i][0], dimentions[i][1], dimentions[i][2], dimentions[i][3] 

                # Colour for current bounding box
                colour_box_current = colours[dimentions[i][4]].tolist()

                # Drawing bounding box on the original current frame
                Box_Rev_Scaler= 1/Box_Scaler
                x_min= round(x_min + box_width*(1-Box_Rev_Scaler)/2)
                y_min= round(y_min + box_height*(1-Box_Rev_Scaler)/2)
                box_width= round(box_width*Box_Rev_Scaler)
                box_height= round(box_height*Box_Rev_Scaler)
                cv2.rectangle(frame_copy, (x_min, y_min),
                                (x_min + box_width, y_min + box_height),
                                colour_box_current, 2)

                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: [{:.3f}]({:.3f})'.format(labels[prediction], 
                                                        scores[i][prediction],
                                                        confidences[i])

                # Putting text with label and confidence on the original image
                cv2.putText(frame_copy, text_box_current, (x_min - 50 , y_min - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.45, colour_box_current, 2)



    # Printing FPS
    cv2.putText(frame_copy, str(round(ping)), (5 , 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, [51, 204, 51], 3)

    # Display the resulting frame. The output window resolution is 1280x720
    cv2.imshow('Video', cv2.resize(frame_copy, (1280, 720)))

    
    # Initializing writer only once
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file. The output video resolution is 1920x1080
        writer = cv2.VideoWriter(Result_Video_Name +'.mp4', fourcc, 20,
                                 (1920, 1080), True)

    # Write processed current frame to the file. The output video resolution is 1920x1080
    writer.write(cv2.resize(frame_copy, (1920, 1080)))

    end2 = time.time()
    t2 += end2 - start

    # Press Q to close the reslut window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Releasing video reader and writer and open windows
video.release()
writer.release()
cv2.destroyAllWindows()

### Mode 3: 
Video playback (Using the Application on an already recoded video) with video saving

In [ ]:
# MODE 3 Cell

# Fill Video input and output name
Input_Video_Name= "FILL ME.mp4"
Input_Video_FPS= 30 #FIX ME
Result_Video_Name= "FILL ME"

%matplotlib inline

# Setting the input video
video = cv2.VideoCapture(Input_Video_Name)

# Writer that will be used to write processed frames
writer = None

# Variables for spatial dimensions of the frames
h, w = None, None

# Setting default size of plots
plt.rcParams['figure.figsize'] = (3, 3)

# Variable for counting total amount of frames
f = 0

# Variables for counting processing time
t = 0
t2 = 0
t_ping_prev=0

# Warmup
frame = np.random.randint(255, size=(416, 416, 3), dtype=np.uint8)
blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
for i in range(3):
    network.setInput(blob)
    detections = network.forward(layers_names_output)


# Catching frames in the loop
while True:
    # Capturing frames one-by-one
    ret, frame = video.read()

    #Calculating FPS
    t_ping= time.time()
    ping= 1/(t_ping - t_ping_prev)
    t_ping_prev= t_ping

    # If the frame was not retrieved
    if not ret:
        break
       
    # Getting spatial dimensions of the frame for the first time
    if w is None or h is None:
        # Slicing two elements from tuple
        h, w = frame.shape[:2]

    # Blob from current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # Forward pass with blob through output layers
    network.setInput(blob)
    start = time.time()
    output_from_network = network.forward(layers_names_output)
    end = time.time()

    # Increasing counters
    f += 1
    t += end - start

    # Lists for detected bounding boxes, confidences and class's number
    bounding_boxes = []
    confidences = []
    class_numbers = []

    # Going through all output layers after feed forward pass
    for result in output_from_network:
        # Filtring Low probability predictions with numpy function to optimize performance
        # 1st order Optimazation
        opt= result[:,5:]
        opt= np.max(opt, axis=1)
        opt2= np.where(opt> probability_minimum )[0]
        # Going through all detections after the optimazation from current output layer 
        for obj in opt2:
            detected_objects= result[obj]
            confidence_current= opt[obj]
            class_current= np.where(detected_objects==confidence_current)[0][0]-5

            # Scaling bounding box coordinates to the initial frame size
            box_current = detected_objects[0:4] * np.array([w, h, w, h])

            # Getting top left corner coordinates
            x_center, y_center, box_width, box_height = box_current
            # Scaler to tune box size with precision
            Box_Scaler= 1.25
            box_width= box_width * Box_Scaler 
            box_height= box_height * Box_Scaler 
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            # Adding results into prepared lists
            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)
                


    # Implementing non-maximum suppression of given bounding boxes
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

    # Genarating a copy from the frame in order to print the labels on this copy
    frame_copy= frame.copy()

    # Checking if there is any detected object been left
    if len(results) > 0:
        dimentions=[]
        # Variable for counting the number of True Signs in the frame
        nbre=0
        # Array containing the signs that will be fed to the classification model
        to_classify= np.array([])
        # Going through indexes of results
        for i in results.flatten():
            # Bounding box coordinates, its width and height
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]
            
            # Cut fragment with Traffic Sign
            c_ts = frame[y_min:y_min+int(box_height), x_min:x_min+int(box_width), :]
            # print(c_ts.shape)
            
            if c_ts.shape[:1] == (0,) or c_ts.shape[1:2] == (0,):
                pass
            else:
                nbre+=1
                # Saving the information about the signs in a list
                dimentions.append((x_min, y_min, box_width, box_height, class_numbers[i]))

                # Getting preprocessed blob with Traffic Sign of needed shape
                blob_ts = cv2.dnn.blobFromImage(c_ts, 1 / 255.0, size=(32, 32), swapRB=True, crop=False)
                blob_ts = blob_ts.transpose(0, 2, 3, 1)

                # Appending the Signs-to-classify array 
                if nbre==1:
                    to_classify= blob_ts
                else:
                    to_classify= np.concatenate((to_classify, blob_ts), axis=0)
                

        if nbre!=0:
            # Obtaing the classification result
            # 2nd order optimazation
            scores= model(to_classify, training=False)

            # Itirating the final predictions in order to print the results 
            for i in range(nbre):

                # Scores is given for image with 43 numbers of predictions for each class
                # Getting only one class with maximum value
                prediction = np.argmax(scores[i])

                # Reobtaing the sign dimentions 
                x_min, y_min, box_width, box_height= dimentions[i][0], dimentions[i][1], dimentions[i][2], dimentions[i][3] 

                # Colour for current bounding box
                colour_box_current = colours[dimentions[i][4]].tolist()

                # Drawing bounding box on the original current frame
                Box_Rev_Scaler= 1/Box_Scaler
                x_min= round(x_min + box_width*(1-Box_Rev_Scaler)/2)
                y_min= round(y_min + box_height*(1-Box_Rev_Scaler)/2)
                box_width= round(box_width*Box_Rev_Scaler)
                box_height= round(box_height*Box_Rev_Scaler)
                cv2.rectangle(frame_copy, (x_min, y_min),
                                (x_min + box_width, y_min + box_height),
                                colour_box_current, 2)

                # Preparing text with label and confidence for current bounding box
                text_box_current = '{}: [{:.3f}]({:.3f})'.format(labels[prediction], 
                                                        scores[i][prediction],
                                                        confidences[i])

                # Putting text with label and confidence on the original image
                cv2.putText(frame_copy, text_box_current, (x_min - 50 , y_min - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.45, colour_box_current, 2)



    # Printing FPS
    cv2.putText(frame_copy, str(round(ping)), (5 , 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, [51, 204, 51], 3)

    # Display the resulting frame. The output window resolution is 1280x720
    cv2.imshow('Video', cv2.resize(frame_copy, (1280, 720)))

    
    # Initializing writer only once
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        # Writing current processed frame into the video file. The output video resolution is 1920x1080
        writer = cv2.VideoWriter(Result_Video_Name +'.mp4', fourcc, Input_Video_FPS,
                                 (1920, 1080), True)

    # Write processed current frame to the file. The output video resolution is 1920x1080
    writer.write(cv2.resize(frame_copy, (1920, 1080)))

    end2 = time.time()
    t2 += end2 - start

    # Press Q to close the reslut window
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Releasing video reader and writer and open windows
video.release()
writer.release()
cv2.destroyAllWindows()

## Some info about the rime and FPS of the result

In [ ]:
print('Total number of frames', f)
print('Total amount of time taken by OpenCV {:.5f} seconds'.format(t))
print('OpenCV FPS:', round((f / t), 1))
print('Total amount of time after the writing step {:.5f} seconds'.format(t2))
print('Code FPS:', round((f / t2), 1))